In [8]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam

In [9]:
data = pd.read_csv('CF_data.csv')
data

,Company,Time,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share,Price
0,YUM,2015-12-01,1283.0,1260.0,-199.0,-1089.0,-28.0,402.4,498.0,1.88,52.53
1,VAR,2010-10-01,360.4,460.8,-74.8,-422.2,-33.3,360.7,363.3,3.23,55.98
2,HRL,2015-10-01,686.1,992.0,-900.9,-70.6,13.1,826.2,834.4,1.61,33.77
3,EPD,2010-12-01,320.8,2300.0,-3251.6,961.1,10.2,-338.8,124.9,0.47,20.80
4,EONGY,2010-12-01,7848.5,14864.2,1013.7,-13307.4,2592.0,-1843.9,606.7,2.24,30.41
5,CF,2012-12-01,1848.7,2375.6,-513.5,-796.8,1067.9,1616.9,1701.4,5.80,40.63
6,BDX,2014-09-01,1185.0,1746.0,-948.0,-807.0,-29.0,949.6,1034.0,5.65,113.81
7,BIIB,2013-12-01,1862.3,2345.1,-1604.7,-716.5,31.9,-1067.7,-1047.8,-4.91,257.51
8,MTRN,2010-12-01,46.4,31.0,-46.5,19.8,3.9,-27.4,-26.3,-1.12,38.64
9,NOC,2010-12-01,2053.0,2453.0,-760.0,-1266.0,427.0,-637.3,-469.1,6.29,58.77


In [10]:
prices = data['Price'] 
data = data.drop(['Company', 'Time', 'Price'], axis = 1)
data

,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share
0,1283.0,1260.0,-199.0,-1089.0,-28.0,402.4,498.0,1.88
1,360.4,460.8,-74.8,-422.2,-33.3,360.7,363.3,3.23
2,686.1,992.0,-900.9,-70.6,13.1,826.2,834.4,1.61
3,320.8,2300.0,-3251.6,961.1,10.2,-338.8,124.9,0.47
4,7848.5,14864.2,1013.7,-13307.4,2592.0,-1843.9,606.7,2.24
5,1848.7,2375.6,-513.5,-796.8,1067.9,1616.9,1701.4,5.80
6,1185.0,1746.0,-948.0,-807.0,-29.0,949.6,1034.0,5.65
7,1862.3,2345.1,-1604.7,-716.5,31.9,-1067.7,-1047.8,-4.91
8,46.4,31.0,-46.5,19.8,3.9,-27.4,-26.3,-1.12
9,2053.0,2453.0,-760.0,-1266.0,427.0,-637.3,-469.1,6.29


In [11]:
col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price', prices)
data

Net Income
Done
Cash from Operations
Done
Cash from Investing
Done
Cash from Financing
Done
Net Change in Cash
Done
Levered Free Cash Flow
Done
Unlevered Free Cash Flow
Done
Free Cash Flow / Share
Done


,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share,Price
0,0.195162,0.101904,0.584679,0.668242,0.474163,0.459175,0.452369,0.003819,52.53
1,0.187545,0.092933,0.585543,0.673199,0.474108,0.458964,0.451685,0.003884,55.98
2,0.190234,0.098895,0.579795,0.675813,0.474591,0.461320,0.454078,0.003806,33.77
3,0.187218,0.113578,0.563438,0.683483,0.474561,0.455424,0.450474,0.003751,20.80
4,0.249369,0.254616,0.593118,0.577408,0.501464,0.447806,0.452921,0.003837,30.41
5,0.199833,0.114427,0.582491,0.670415,0.485583,0.465322,0.458481,0.004008,40.63
6,0.194353,0.107359,0.579467,0.670339,0.474153,0.461945,0.455091,0.004001,113.81
7,0.199945,0.114084,0.574898,0.671011,0.474787,0.451734,0.444518,0.003492,257.51
8,0.184952,0.088108,0.585740,0.676485,0.474495,0.457000,0.449706,0.003675,38.64
9,0.201519,0.115296,0.580776,0.666926,0.478904,0.453913,0.447457,0.004032,58.77


In [12]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:8] # Parameters
y = NpMatrix[:,8] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))
number_of_features = len(X[1])

X: [[0.19516198 0.10190382 0.58467919 ... 0.45917516 0.45236896 0.00381927]
 [0.18754474 0.09293251 0.58554341 ... 0.4589641  0.45168483 0.00388429]
 [0.19023381 0.09889542 0.57979515 ... 0.46132016 0.4540775  0.00380627]
 ...
 [0.19813425 0.10176912 0.58603606 ... 0.45923792 0.45299671 0.00382891]
 [0.20573002 0.12755377 0.58208374 ... 0.45472016 0.44804477 0.00381687]
 [0.19787004 0.0888824  0.57738688 ... 0.46291752 0.45705425 0.00369453]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 8
Total Number of Training instances: 5000


[52.53 55.98 33.77 ... 63.33 53.15 40.29]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 52.53
Total number of y values 5000


In [13]:
#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=8, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.3))
    
    network.add(layers.Dense(units=8, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.3))

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=1, activation='relu'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [14]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 1000, 
                                 batch_size=5)

In [15]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 4000 samples
Epoch 1/1000
4000/4000 [==============================] - 3s 845us/sample - loss: 65.6648 - mae: 65.6648 - mse: 23941.8477
Epoch 2/1000
4000/4000 [==============================] - 1s 372us/sample - loss: 55.4526 - mae: 55.4526 - mse: 21692.0371
Epoch 3/1000
4000/4000 [==============================] - 1s 370us/sample - loss: 53.4074 - mae: 53.4074 - mse: 21457.9902
Epoch 4/1000
4000/4000 [==============================] - 2s 415us/sample - loss: 52.2482 - mae: 52.2482 - mse: 21274.6289
Epoch 5/1000
4000/4000 [==============================] - 2s 470us/sample - loss: 51.8551 - mae: 51.8552 - mse: 20939.6543
Epoch 6/1000
4000/4000 [==============================] - 2s 395us/sample - loss: 51.2872 - mae: 51.2872 - mse: 21086.2207
Epoch 7/1000
4000/4000 [==============================] - 1s 365us/sample - loss: 51.1732 - mae: 51.1732 - mse: 20889.9375
Epoch 8/1000
4000/4000 [==============================] - 2s 426us/sample - loss: 51.0550 - mae: 51.0550 - mse: 21028

Epoch 67/1000
4000/4000 [==============================] - 2s 595us/sample - loss: 48.8605 - mae: 48.8605 - mse: 20587.5781
Epoch 68/1000
4000/4000 [==============================] - 2s 396us/sample - loss: 48.9914 - mae: 48.9914 - mse: 20503.5391
Epoch 69/1000
4000/4000 [==============================] - 1s 362us/sample - loss: 48.6205 - mae: 48.6205 - mse: 20569.5820
Epoch 70/1000
4000/4000 [==============================] - 1s 326us/sample - loss: 48.8271 - mae: 48.8270 - mse: 20593.2246
Epoch 71/1000
4000/4000 [==============================] - 2s 380us/sample - loss: 48.4774 - mae: 48.4774 - mse: 20514.1133
Epoch 72/1000
4000/4000 [==============================] - 1s 318us/sample - loss: 48.7174 - mae: 48.7174 - mse: 20494.3848
Epoch 73/1000
4000/4000 [==============================] - 1s 329us/sample - loss: 48.7820 - mae: 48.7820 - mse: 20554.1855
Epoch 74/1000
4000/4000 [==============================] - 1s 328us/sample - loss: 48.4893 - mae: 48.4893 - mse: 20564.0801
Epoch 75

4000/4000 [==============================] - 3s 658us/sample - loss: 48.4732 - mae: 48.4732 - mse: 20473.5371
Epoch 133/1000
4000/4000 [==============================] - 2s 475us/sample - loss: 48.3439 - mae: 48.3439 - mse: 20472.0938
Epoch 134/1000
4000/4000 [==============================] - 1s 329us/sample - loss: 48.5131 - mae: 48.5131 - mse: 20538.2754
Epoch 135/1000
4000/4000 [==============================] - 1s 343us/sample - loss: 48.3202 - mae: 48.3201 - mse: 20516.4609
Epoch 136/1000
4000/4000 [==============================] - 1s 367us/sample - loss: 48.4889 - mae: 48.4889 - mse: 20549.9961
Epoch 137/1000
4000/4000 [==============================] - 1s 343us/sample - loss: 48.3576 - mae: 48.3576 - mse: 20498.4805
Epoch 138/1000
4000/4000 [==============================] - 1s 374us/sample - loss: 48.4517 - mae: 48.4516 - mse: 20506.8809
Epoch 139/1000
4000/4000 [==============================] - 1s 338us/sample - loss: 48.2274 - mae: 48.2274 - mse: 20558.7852
Epoch 140/1000


4000/4000 [==============================] - 1s 311us/sample - loss: 48.0456 - mae: 48.0456 - mse: 20416.5566
Epoch 198/1000
4000/4000 [==============================] - 1s 307us/sample - loss: 48.2353 - mae: 48.2353 - mse: 20487.0254
Epoch 199/1000
4000/4000 [==============================] - 1s 322us/sample - loss: 48.3004 - mae: 48.3003 - mse: 20535.2520
Epoch 200/1000
4000/4000 [==============================] - 1s 360us/sample - loss: 48.0431 - mae: 48.0431 - mse: 20475.0547
Epoch 201/1000
4000/4000 [==============================] - 1s 314us/sample - loss: 48.0675 - mae: 48.0675 - mse: 20473.7246
Epoch 202/1000
4000/4000 [==============================] - 1s 329us/sample - loss: 48.1449 - mae: 48.1449 - mse: 20486.7812
Epoch 203/1000
4000/4000 [==============================] - 1s 316us/sample - loss: 48.0330 - mae: 48.0330 - mse: 20481.4941
Epoch 204/1000
4000/4000 [==============================] - 1s 359us/sample - loss: 48.0434 - mae: 48.0434 - mse: 20438.5801
Epoch 205/1000


4000/4000 [==============================] - 2s 436us/sample - loss: 47.8742 - mae: 47.8742 - mse: 20477.1602
Epoch 263/1000
4000/4000 [==============================] - 1s 336us/sample - loss: 47.8066 - mae: 47.8066 - mse: 20415.3379
Epoch 264/1000
4000/4000 [==============================] - 2s 400us/sample - loss: 47.9223 - mae: 47.9223 - mse: 20430.8145
Epoch 265/1000
4000/4000 [==============================] - 1s 333us/sample - loss: 47.9012 - mae: 47.9011 - mse: 20459.4453
Epoch 266/1000
4000/4000 [==============================] - 2s 386us/sample - loss: 47.9277 - mae: 47.9277 - mse: 20431.1230
Epoch 267/1000
4000/4000 [==============================] - 2s 395us/sample - loss: 47.7121 - mae: 47.7121 - mse: 20375.3008
Epoch 268/1000
4000/4000 [==============================] - 1s 341us/sample - loss: 47.7964 - mae: 47.7964 - mse: 20460.3887
Epoch 269/1000
4000/4000 [==============================] - 2s 415us/sample - loss: 47.8640 - mae: 47.8640 - mse: 20520.4434
Epoch 270/1000


...

4000/4000 [==============================] - 1s 327us/sample - loss: 48.3768 - mae: 48.3768 - mse: 21068.1113
Epoch 717/1000
4000/4000 [==============================] - 1s 348us/sample - loss: 48.1786 - mae: 48.1787 - mse: 21073.6621
Epoch 718/1000
4000/4000 [==============================] - 1s 354us/sample - loss: 48.2355 - mae: 48.2355 - mse: 21055.1582
Epoch 719/1000
4000/4000 [==============================] - 2s 389us/sample - loss: 48.0670 - mae: 48.0670 - mse: 21017.8828
Epoch 720/1000
4000/4000 [==============================] - 1s 325us/sample - loss: 48.2328 - mae: 48.2327 - mse: 21042.1289
Epoch 721/1000
4000/4000 [==============================] - 1s 342us/sample - loss: 48.2154 - mae: 48.2154 - mse: 21049.1523
Epoch 722/1000
4000/4000 [==============================] - 2s 417us/sample - loss: 48.2254 - mae: 48.2254 - mse: 21080.8242
Epoch 723/1000
4000/4000 [==============================] - 2s 412us/sample - loss: 48.0984 - mae: 48.0984 - mse: 21043.1113
Epoch 724/1000


4000/4000 [==============================] - 1s 361us/sample - loss: 48.2460 - mae: 48.2460 - mse: 21019.7480
Epoch 782/1000
4000/4000 [==============================] - 1s 371us/sample - loss: 48.1689 - mae: 48.1689 - mse: 21023.2070
Epoch 783/1000
4000/4000 [==============================] - 2s 379us/sample - loss: 48.0781 - mae: 48.0781 - mse: 21011.7871
Epoch 784/1000
4000/4000 [==============================] - 2s 399us/sample - loss: 48.1159 - mae: 48.1159 - mse: 20995.5723
Epoch 785/1000
4000/4000 [==============================] - 1s 363us/sample - loss: 48.1162 - mae: 48.1161 - mse: 20979.8379
Epoch 786/1000
4000/4000 [==============================] - 2s 383us/sample - loss: 48.1499 - mae: 48.1498 - mse: 20997.6289- loss: 35.
Epoch 787/1000
4000/4000 [==============================] - 1s 352us/sample - loss: 48.2406 - mae: 48.2406 - mse: 21051.2949
Epoch 788/1000
4000/4000 [==============================] - 2s 396us/sample - loss: 48.1201 - mae: 48.1201 - mse: 21018.5859s - l

4000/4000 [==============================] - 1s 371us/sample - loss: 48.1555 - mae: 48.1554 - mse: 21042.9609
Epoch 847/1000
4000/4000 [==============================] - 1s 265us/sample - loss: 48.2192 - mae: 48.2192 - mse: 21018.7773
Epoch 848/1000
4000/4000 [==============================] - 1s 278us/sample - loss: 48.1082 - mae: 48.1082 - mse: 20964.8633
Epoch 849/1000
4000/4000 [==============================] - 1s 264us/sample - loss: 48.2564 - mae: 48.2564 - mse: 21060.0605
Epoch 850/1000
4000/4000 [==============================] - 1s 291us/sample - loss: 48.0608 - mae: 48.0608 - mse: 20978.3828
Epoch 851/1000
4000/4000 [==============================] - 1s 281us/sample - loss: 48.2690 - mae: 48.2690 - mse: 21066.4121
Epoch 852/1000
4000/4000 [==============================] - 1s 259us/sample - loss: 48.2290 - mae: 48.2290 - mse: 21029.6641
Epoch 853/1000
4000/4000 [==============================] - 1s 288us/sample - loss: 48.2267 - mae: 48.2267 - mse: 21075.4707
Epoch 854/1000


4000/4000 [==============================] - 1s 279us/sample - loss: 48.2123 - mae: 48.2124 - mse: 21006.4980
Epoch 912/1000
4000/4000 [==============================] - 1s 262us/sample - loss: 48.2384 - mae: 48.2383 - mse: 21025.2617
Epoch 913/1000
4000/4000 [==============================] - 1s 280us/sample - loss: 48.1192 - mae: 48.1192 - mse: 20975.4121- loss: 46.7146 - mae: 46.714
Epoch 914/1000
4000/4000 [==============================] - 1s 287us/sample - loss: 48.1683 - mae: 48.1683 - mse: 21039.6191
Epoch 915/1000
4000/4000 [==============================] - 1s 263us/sample - loss: 48.1473 - mae: 48.1473 - mse: 20997.9727
Epoch 916/1000
4000/4000 [==============================] - 1s 291us/sample - loss: 48.0759 - mae: 48.0759 - mse: 20999.8418
Epoch 917/1000
4000/4000 [==============================] - 1s 262us/sample - loss: 48.2785 - mae: 48.2785 - mse: 21079.3047
Epoch 918/1000
4000/4000 [==============================] - 1s 289us/sample - loss: 48.2582 - mae: 48.2582 - ms

4000/4000 [==============================] - 1s 260us/sample - loss: 48.2033 - mae: 48.2033 - mse: 21018.3613
Epoch 976/1000
4000/4000 [==============================] - 1s 293us/sample - loss: 48.1735 - mae: 48.1735 - mse: 21021.4688
Epoch 977/1000
4000/4000 [==============================] - 1s 274us/sample - loss: 48.2184 - mae: 48.2184 - mse: 21031.4023
Epoch 978/1000
4000/4000 [==============================] - 1s 281us/sample - loss: 48.0762 - mae: 48.0762 - mse: 20953.2188
Epoch 979/1000
4000/4000 [==============================] - 1s 284us/sample - loss: 48.1572 - mae: 48.1572 - mse: 21045.7930
Epoch 980/1000
4000/4000 [==============================] - 1s 266us/sample - loss: 48.1869 - mae: 48.1869 - mse: 21028.8984
Epoch 981/1000
4000/4000 [==============================] - 1s 303us/sample - loss: 48.1370 - mae: 48.1370 - mse: 21003.3359
Epoch 982/1000
4000/4000 [==============================] - 1s 262us/sample - loss: 48.0597 - mae: 48.0597 - mse: 20956.7598
Epoch 983/1000


In [16]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-46.70371597 -44.48901157 -51.95150793 -48.39398563 -43.91425868]
Mean MAE: -47.09049595689773
Standard Deviation of the mean: 2.9097878175894203
